<a href="https://colab.research.google.com/github/Nathan-Levy/HCS-Gaze-Detection/blob/main/Copy_of_Downtown_LA_Traffic_Prediction_I110_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow import keras

In [ ]:
controller_data_test = pd.read_csv('Combined flows test.csv')
controller_data_test.dropna(how='all', inplace=True)

controller_data_train = pd.read_csv('Combined flows train.csv')
controller_data_train.dropna(how='all', inplace=True)

In [ ]:
controller_data_train["5 Minutes"] = pd.to_datetime(controller_data_train["5 Minutes"])
controller_data_train.set_index("5 Minutes", inplace=True)

# Keep only the relevant column
time_flow_train = controller_data_train[['MADOR Flow (Veh/5 Minutes)', 'THIRD Flow (Veh/5 Minutes)', '11TH Flow (Veh/5 Minutes)', 'ADAMS Flow (Veh/5 Minutes)', 'KING BLVD Flow (Veh/5 Minutes)']]

controller_data_test["5 Minutes"] = pd.to_datetime(controller_data_test["5 Minutes"])
controller_data_test.set_index("5 Minutes", inplace=True)

# Keep only the relevant column
time_flow_test = controller_data_test[['MADOR Flow (Veh/5 Minutes)', 'THIRD Flow (Veh/5 Minutes)', '11TH Flow (Veh/5 Minutes)', 'ADAMS Flow (Veh/5 Minutes)', 'KING BLVD Flow (Veh/5 Minutes)']]

print(time_flow_test)

                     MADOR Flow (Veh/5 Minutes)  THIRD Flow (Veh/5 Minutes)  \
5 Minutes                                                                     
2018-06-04 00:00:00                         116                         179   
2018-06-04 00:05:00                         119                         168   
2018-06-04 00:10:00                          88                         141   
2018-06-04 00:15:00                          81                         130   
2018-06-04 00:20:00                          84                         166   
...                                         ...                         ...   
2018-07-01 23:35:00                         153                         143   
2018-07-01 23:40:00                         152                         139   
2018-07-01 23:45:00                         139                         136   
2018-07-01 23:50:00                         149                         132   
2018-07-01 23:55:00                         136     

In [ ]:
# Normalise
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
flow_train_scaled = scaler.fit_transform(time_flow_train)
flow_test_scaled = scaler.transform(time_flow_test)

In [ ]:
# Train and validation split
flow_train, flow_test, = flow_train_scaled, flow_test_scaled

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Define look-back values to test (1 hour, 1 day, 3 days, 1 week)
look_back_values = [12, 288, 864, 2016]

# Dictionary to store evaluation results
results = {}

# Function to create dataset for prediction
def create_dataset(dataset, look_back):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), :])  # Past 'look_back' time steps as input
        y.append(dataset[i + look_back, :])  # Predict next time step
    return np.array(X), np.array(y)

# Iterate over different look-back values
for look_back in look_back_values:
    print(f"\nTraining model with look_back = {look_back}...")

    # Prepare dataset
    X_train, y_train = create_dataset(flow_train, look_back)
    X_test, y_test = create_dataset(flow_test, look_back)

    # Reshape for LSTM input
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

    input_layer = keras.layers.Input(shape=(look_back, 5))
    encoded = keras.layers.LSTM(64, activation='tanh', return_sequences=True)(input_layer)
    encoded = keras.layers.LSTM(32, activation='tanh', return_sequences=False)(encoded)

    decoded = keras.layers.RepeatVector(look_back)(encoded)
    decoded = keras.layers.LSTM(64, activation='tanh', return_sequences=True)(decoded)
    decoded = keras.layers.TimeDistributed(layers.Dense(5, activation='linear'))(decoded)

    # Define autoencoder
    autoencoder = keras.models.Model(inputs=input_layer, outputs=decoded)

    # Compile autoencoder
    optimizer = keras.optimizers.Adam(clipnorm=1.0, learning_rate=0.0005)
    autoencoder.compile(optimizer=optimizer, loss='mse')

    # Train autoencoder
    autoencoder.fit(X_train, X_train, epochs=20, batch_size=16, validation_data=(X_test, X_test),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

    # Extract the encoder
    encoder = keras.models.Model(inputs=input_layer, outputs=encoded)

    # Generate encoded features
    encoded_features_train = encoder.predict(X_train)
    encoded_features_test = encoder.predict(X_test)

    prediction_model = keras.models.Sequential([
        keras.layers.Dense(16, activation='relu', input_dim=encoded_features_train.shape[1]),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(5, activation='linear')  # Predicting next time step
    ])

    # Compile predictor
    prediction_model.compile(optimizer='adam', loss='mse')

    # Train predictor
    prediction_model.fit(encoded_features_train, y_train, epochs=20, batch_size=16, validation_data=(encoded_features_test, y_test),
                         callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

    # Make predictions
    y_pred = prediction_model.predict(encoded_features_test)

    # Inverse transform predictions (if data was normalized)
    y_pred_original = scaler.inverse_transform(y_pred)
    y_test_original = scaler.inverse_transform(y_test)

    # Step 3: Compute Evaluation Metrics
    def mean_absolute_error(y_true, y_pred):
        return np.mean(np.abs(y_true - y_pred))

    def mean_relative_error(y_true, y_pred):
        return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100  # Percentage

    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_true - y_pred) ** 2))

    mae = mean_absolute_error(y_test_original, y_pred_original)
    mre = mean_relative_error(y_test_original, y_pred_original)
    rmse = root_mean_squared_error(y_test_original, y_pred_original)

    # Store results
    results[look_back] = {'MAE': mae, 'MRE': mre, 'RMSE': rmse}

    print(f"Look-Back: {look_back} → MAE: {mae:.4f}, MRE: {mre:.2f}%, RMSE: {rmse:.4f}")

best_look_back = min(results, key=lambda x: results[x]['MAE'])  # Choose based on MAE
print(f"\nOptimal Look-Back: {best_look_back} with lowest MAE: {results[best_look_back]['MAE']:.4f}")



Training model with look_back = 12...
Epoch 1/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - loss: 0.0231 - val_loss: 0.0034
Epoch 2/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0023 - val_loss: 0.0024
Epoch 3/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0018 - val_loss: 0.0022
Epoch 4/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0017 - val_loss: 0.0021
Epoch 5/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0017 - val_loss: 0.0019
Epoch 6/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0016 - val_loss: 0.0019
Epoch 7/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0015 - val_loss: 0.0018
Epoch 8/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0015 - val_loss: 0.0018
Epoch 9/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 10/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0014 - val_loss: 0.0017
Epoch 11/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1134/1134 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0188 - val_loss: 0.0041
Epoch 2/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0029 - val_loss: 0.0036
Epoch 3/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0025 - val_loss: 0.0031
Epoch 4/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0024 - val_loss: 0.0031
Epoch 5/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0024 - val_loss: 0.0029
Epoch 6/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0024 - val_loss: 0.0030
Epoch 7/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0023 - val_loss: 0.0030
Epoch 8/20
1134/1134 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0023 - val_loss: 0.0031
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Look-Back: 12 → MAE: 30.1703, MRE: 8.93%, RMSE: 41.6570

Training model with look_back = 288...
Epoch 1/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 71s 53ms/step - loss: 0.0466 - val_loss: 0.0392
Epoch 2/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 56s 51ms/step - loss: 0.0359 - val_los

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1116/1116 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0848 - val_loss: 0.0069
Epoch 2/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0044 - val_loss: 0.0065
Epoch 3/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0042 - val_loss: 0.0063
Epoch 4/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0040 - val_loss: 0.0060
Epoch 5/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0037 - val_loss: 0.0063
Epoch 6/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0036 - val_loss: 0.0058
Epoch 7/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0035 - val_loss: 0.0059
Epoch 8/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0035 - val_loss: 0.0060
Epoch 9/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0034 - val_loss: 0.0058
Epoch 10/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0034 - val_loss: 0.0057
Epoch 11/20
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0034 - val_loss: 0.0060
Epoch 12/20
1116/1116 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1080/1080 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0732 - val_loss: 0.0100
Epoch 2/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0071 - val_loss: 0.0084
Epoch 3/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0056 - val_loss: 0.0078
Epoch 4/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0049 - val_loss: 0.0071
Epoch 5/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0047 - val_loss: 0.0071
Epoch 6/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0046 - val_loss: 0.0071
Epoch 7/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0045 - val_loss: 0.0067
Epoch 8/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0044 - val_loss: 0.0067
Epoch 9/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0043 - val_loss: 0.0065
Epoch 10/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0043 - val_loss: 0.0063
Epoch 11/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0042 - val_loss: 0.0066
Epoch 12/20
1080/1080 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1008/1008 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0944 - val_loss: 0.0088
Epoch 2/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0055 - val_loss: 0.0078
Epoch 3/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0051 - val_loss: 0.0075
Epoch 4/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0049 - val_loss: 0.0076
Epoch 5/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0048 - val_loss: 0.0070
Epoch 6/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0047 - val_loss: 0.0070
Epoch 7/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0046 - val_loss: 0.0070
Epoch 8/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0045 - val_loss: 0.0066
Epoch 9/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0043 - val_loss: 0.0068
Epoch 10/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0042 - val_loss: 0.0068
Epoch 11/20
1008/1008 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0042 - val_loss: 0.0064
Epoch 12/20
1008/1008 ━━━━━━━━━━━━━━━━━━━